## Célula 0: Instalar Dependências

In [ ]:
# ==============================
# INSTALAÇÃO DE BIBLIOTECAS
# ==============================
import sys
!{sys.executable} -m pip install requests beautifulsoup4 pandas plotly jinja2 --quiet

print("✅ Bibliotecas instaladas! Reinicie o kernel se necessário (Kernel → Restart).")

## Importações e Configurações

In [1]:
# ==============================================================
# CGI.br Scraper - Versão Notebook (Independente)
# ==============================================================
# Coleta notícias e atas de reuniões do CGI.br
# Mostra dados na hora, gráficos interativos na célula, exporta CSV e banco SQLite local
# ==============================================================

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import sqlite3
import plotly.express as px
import plotly.io as pio
import time

# Gráficos interativos na célula
pio.renderers.default = "notebook_connected"

# Banco local
DATABASE_NAME = 'cgi_br_news.db'

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
}

print("✅ Pronto para scraping CGI.br!")

✅ Pronto para scraping CGI.br!


## Scraper Notícias CGI.br

In [ ]:
# ==============================
# SCRAPER NOTÍCIAS CGI.br (PAGINAÇÃO COM page:N)
# ==============================

def scrape_cgi_noticias(max_pages=20):  # Ajuste para mais (ex: 50 se tiver muitas)
    print("🔄 Scraping Notícias CGI.br (paginação com page:N)...")
    start_time = time.time()
    articles = []
    base_url = "https://cgi.br/noticias/indice/"
    
    for page in range(1, max_pages + 1):
        if page == 1:
            url = base_url
        else:
            url = f"https://cgi.br/noticias/indice/page:{page}"
        
        print(f"   📄 Página {page}: {url}")
        
        try:
            response = requests.get(url, headers=HEADERS, timeout=30)
            if response.status_code == 404:
                print(f"      ⚠️ Página {page} não existe (404). Parando aqui.")
                break
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            news_items = soup.find_all('article', class_='row blog-post')
            
            if len(news_items) == 0:
                print(f"      ⚠️ Nenhuma notícia na página {page}. Parando.")
                break
                
            print(f"      🔍 {len(news_items)} notícias encontradas.")
            
            for item in news_items:
                title_tag = item.find('h2')
                title = title_tag.get_text(strip=True) if title_tag else 'N/A'
                
                link_tag = title_tag.find('a') if title_tag else None
                link = "https://cgi.br" + link_tag['href'] if link_tag else 'N/A'
                
                date_tag = item.find('span', class_='date-post')
                date_str = date_tag.get_text(strip=True).upper() if date_tag else 'N/A'
                try:
                    date = datetime.strptime(date_str, '%d %b %Y').strftime('%Y-%m-%d')
                except:
                    date = date_str
                
                articles.append({
                    'title': title,
                    'date': date,
                    'author': 'CGI.br Notícias',
                    'url': link,
                    'source': 'CGI Notícias'
                })
            
            time.sleep(1.2)  # Respeito ao servidor
            
        except Exception as e:
            print(f"   ❌ Erro na página {page}: {e}")
            break
    
    df = pd.DataFrame(articles)
    print(f"\n✅ {len(df)} notícias coletadas de {page} páginas!")
    display(df.head(20).style.set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'}))
    exec_time = time.time() - start_time
    print(f"⏱️ Tempo total: {exec_time:.2f}s")
    return df

#vcs podem ajustar conforme o numero de paginas que voces querem fazer a raspagem
df_noticias = scrape_cgi_noticias(max_pages=20)

🔄 Scraping Notícias CGI.br (paginação com page:N)...
   📄 Página 1: https://cgi.br/noticias/indice/
      🔍 10 notícias encontradas.
   📄 Página 2: https://cgi.br/noticias/indice/page:2
      🔍 10 notícias encontradas.
   📄 Página 3: https://cgi.br/noticias/indice/page:3
      🔍 10 notícias encontradas.
   📄 Página 4: https://cgi.br/noticias/indice/page:4
      🔍 10 notícias encontradas.
   📄 Página 5: https://cgi.br/noticias/indice/page:5
      🔍 10 notícias encontradas.
   📄 Página 6: https://cgi.br/noticias/indice/page:6
      🔍 10 notícias encontradas.
   📄 Página 7: https://cgi.br/noticias/indice/page:7
      🔍 10 notícias encontradas.
   📄 Página 8: https://cgi.br/noticias/indice/page:8
      🔍 10 notícias encontradas.
   📄 Página 9: https://cgi.br/noticias/indice/page:9
      🔍 10 notícias encontradas.
   📄 Página 10: https://cgi.br/noticias/indice/page:10
      🔍 10 notícias encontradas.
   📄 Página 11: https://cgi.br/noticias/indice/page:11
      🔍 10 notícias encontradas.
   

,title,date,author,url,source
0,CGI.br e NIC.br lançam programa de incentivo à pesquisa sobre impactos da Inteligência Artificial na Web,2025-11-11,CGI.br Notícias,https://cgi.br/noticia/releases/cgi-br-e-nic-br-lancam-programa-de-incentivo-a-pesquisa-sobre-impactos-da-inteligencia-artificial-na-web/,CGI Notícias
1,"Nota de pesar: Rubens Kühl, gerente de produtos e mercado do Registro.br",2025-11-03,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-pesar-rubens-kuhl-gerente-de-produtos-e-mercado-do-registro-br/,CGI Notícias
2,"Note of Condolence: Rubens Kühl, Registro.br Product Marketing Manager",2025-11-03,CGI.br Notícias,https://cgi.br/noticia/notas/note-of-condolence-rubens-kuhl-registro-br-product-marketing-manager/,CGI Notícias
3,"TIC Kids Online Brasil: 65% das crianças e dos adolescentes usam IA generativa para estudar, criar conteúdo e lidar com emoções",22 OUT 2025,CGI.br Notícias,https://cgi.br/noticia/releases/tic-kids-online-brasil-65-das-criancas-e-dos-adolescentes-usam-ia-generativa-para-estudar-criar-conteudo-e-lidar-com-emocoes/,CGI Notícias
4,10º Simpósio de Crianças e Adolescentes na Internet debate ECA Digital e impactos das apostas online no público infantojuvenil,16 OUT 2025,CGI.br Notícias,https://cgi.br/noticia/releases/10-simposio-de-criancas-e-adolescentes-na-internet-debate-eca-digital-e-impactos-das-apostas-online-no-publico-infantojuvenil/,CGI Notícias
5,"Uso de Inteligência Artificial em equipamentos culturais brasileiros é incipiente, mas digitalização avança, revela pesquisa TIC Cultura 2024",07 OUT 2025,CGI.br Notícias,https://cgi.br/noticia/releases/uso-de-inteligencia-artificial-em-equipamentos-culturais-brasileiros-e-incipiente-mas-digitalizacao-avanca-revela-pesquisa-tic-cultura-2024/,CGI Notícias
6,Mais acessível e completo: conheça o novo site do Cetic.br,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/mais-acessivel-e-completo-conheca-o-novo-site-do-cetic-br/,CGI Notícias
7,"Evento em São Paulo celebra 15 anos de SACI-Adm, referência na disputa de domínios de Internet no Brasil",30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/releases/evento-em-sao-paulo-celebra-15-anos-de-saci-adm-referencia-na-disputa-de-dominios-de-internet-no-brasil/,CGI Notícias
8,Carta de Maputo do 3º Fórum Lusófono de Governação da Internet,23 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/carta-de-maputo-do-3-forum-lusofono-de-governacao-da-internet/,CGI Notícias
9,"Nota de pesar: Péter Major, referência global em governança na Internet",19 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-pesar-peter-major-referencia-global-em-governanca-na-internet/,CGI Notícias


⏱️ Tempo total: 34.80s


## Scraper Reuniões CGI.br

In [6]:
# ==============================
# SCRAPER REUNIÕES (SEM INSERT)
# ==============================

def scrape_cgi_reunioes():
    print("Scraping Reuniões Extraordinárias CGI.br...")
    url = "https://www.cgi.br/reunioes/extraordinarias/"
    try:
        response = requests.get(url, headers=HEADERS, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        date_pattern = re.compile(r'\d{2} de [a-zç]+ de \d{4}', re.IGNORECASE)
        date_elements = soup.find_all(string=date_pattern)
        
        articles = []
        for date_elem in date_elements:
            current_date = date_elem.strip()
            parent = date_elem.find_parent('div')
            if not parent:
                continue
            ul = parent.find_next('ul')
            if not ul:
                continue
            
            items = ul.find_all('li')
            for item in items:
                text = item.get_text(strip=True)
                link_tag = item.find('a', string=re.compile(r'ata', re.I))
                link = "https://www.cgi.br" + link_tag['href'] if link_tag else url
                
                articles.append({
                    'title': text,
                    'date': current_date,
                    'author': 'CGI.br Reuniões',
                    'url': link,
                    'source': 'CGI Reuniões'
                })
                
                # INSERT COMENTADO
                # insert_article(text, current_date, 'CGI.br Reuniões', link, 'CGI Reuniões')
        
        df = pd.DataFrame(articles)
        print(f"\n{len(df)} atas de reuniões coletadas!")
        display(df.head(10))
        return df
    except Exception as e:
        print(f"Erro: {e}")
        return pd.DataFrame()

# === RODE AQUI ===
df_reunioes = scrape_cgi_reunioes()

Scraping Reuniões Extraordinárias CGI.br...

30 atas de reuniões coletadas!


,title,date,author,url,source
0,Política de Privacidade do WhatsApp,04 de agosto de 2025,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
1,Política de Privacidade do WhatsApp,13 de setembro de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
2,Política de Privacidade do WhatsApp,09 de julho de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
3,Visite o site do CGI.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
4,Visite o site do NIC.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
5,Visite o site do Registro.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
6,Visite o site do CERT.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
7,Visite o site do CETIC.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
8,Visite o site do CEPTRO.br,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões
9,Visite o site do Ceweb,16 de abril de 2021,CGI.br Reuniões,https://www.cgi.br/reunioes/extraordinarias/,CGI Reuniões


## Combinar Dados + Filtro + Gráficos

In [8]:
# ==============================
# COMBINAR E GRÁFICOS
# ==============================

df_all = pd.concat([df_noticias, df_reunioes], ignore_index=True) if 'df_noticias' in locals() and 'df_reunioes' in locals() else pd.DataFrame()

keywords = ['governança', 'internet', 'digital', 'reunião', 'ata', 'política', 'CGI']

df_filtered = df_all[
    df_all['title'].str.contains('|'.join(keywords), case=False, na=False)
].copy()

print(f"\n{len(df_filtered)} itens filtrados de {len(df_all)} totais.")

def plot_charts(df):
    if df.empty:
        print("Sem dados.")
        return
    
    # Top 15 Títulos
    top15 = df.head(15).copy()
    top15['rank'] = range(1, len(top15) + 1)
    fig1 = px.bar(top15, x='rank', y='title', orientation='h', title='Top 15 Itens')
    fig1.update_layout(height=600, yaxis={'categoryorder': 'array', 'categoryarray': top15['title'][::-1]})
    fig1.show()
    
    # Pizza por Fonte
    source_count = df['source'].value_counts().reset_index()
    fig2 = px.pie(source_count, names='source', values='count', title='Distribuição por Fonte')
    fig2.show()
    
    # Nuvem de Palavras
    text = ' '.join(df['title']).lower()
    words = re.findall(r'\b\w{4,}\b', text)
    word_freq = pd.Series(words).value_counts().head(20).reset_index()
    word_freq.columns = ['palavra', 'freq']
    fig3 = px.treemap(word_freq, path=['palavra'], values='freq', title='Nuvem de Palavras')
    fig3.show()

plot_charts(df_filtered if not df_filtered.empty else df_all)


135 itens filtrados de 230 totais.


## Exportar CSV

In [9]:
# ==============================
# EXPORTAR CSV
# ==============================

timestamp = datetime.now().strftime('%Y%m%d_%H%M')
df_all.to_csv(f"cgi_all_{timestamp}.csv", index=False, encoding='utf-8')
if not df_filtered.empty:
    df_filtered.to_csv(f"cgi_filtered_{timestamp}.csv", index=False, encoding='utf-8')
print("CSVs exportados na pasta atual!")

CSVs exportados na pasta atual!
